# First Iteration with Recurrent Neural Networks and Post Padding

In [1]:
import pandas as pd
import numpy as np
from progress.bar import Bar
#from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical, pad_sequences

2023-10-24 22:02:08.982956: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 22:02:09.447658: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 22:02:09.449808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 22:02:13.254860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv("./Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [8]:
def complete_array_by_repeat(X):
    r = 97-X.shape[0]
    concatenated_array = X
    for i in range(0,r):
        new_row = X[i%X.shape[0], :]
        concatenated_array = np.vstack((new_row, concatenated_array))
    return concatenated_array

In [9]:
X_list = []
y_list = []
with Bar("Processing...", max = df.size) as bar:
    for file_name, class_name in df.iterrows():
        np_element = np.load(f"../Clean_Data/{class_name.iloc[0]}/{file_name}")
        if np_element.shape[0]<97:
            np_element = complete_array_by_repeat(np_element)
        #np_element = np.repeat(np_element,[1,2], axis = 1)
        X_list.append(np_element[:50])#[:33,:])
        y_list.append(class_name.iloc[0])
        bar.next()

In [10]:
if X_list[0].shape[0]<97:
    print(True)

True


In [11]:
len(X_list)

1500

In [12]:
min_arr = 1000
for arr in X_list:
    aux_arr = arr.shape[0]
    if aux_arr < min_arr:
        min_arr = aux_arr
print(min_arr)

50


In [13]:
max_arr = 0
for arr in X_list:
    aux_arr = arr.shape[0]
    if aux_arr > max_arr:
        max_arr = aux_arr
print(max_arr)

50


In [14]:
X = np.stack(X_list)
y = np.array(y_list)

In [15]:
X.shape

(1500, 50, 1662)

In [16]:
y.shape

(1500,)

# GaussianHMM

# Recurrent Neural Networks

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(50,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
y = to_categorical(y)

In [20]:
y.shape

(1500, 6)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [22]:
y_test.shape

(450, 6)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 50, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596774 (2.28 MB)
Trainable params: 596774 

In [25]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
17/17 [==============================] - 6s 373ms/step - loss: 1332.1636 - categorical_accuracy: 0.2229
Epoch 2/50
17/17 [==============================] - 7s 387ms/step - loss: 1227.2812 - categorical_accuracy: 0.2124
Epoch 3/50
17/17 [==============================] - 7s 384ms/step - loss: 680.2358 - categorical_accuracy: 0.2048
Epoch 4/50
17/17 [==============================] - 7s 408ms/step - loss: 1434.7214 - categorical_accuracy: 0.2057
Epoch 5/50
17/17 [==============================] - 7s 446ms/step - loss: 2202.4229 - categorical_accuracy: 0.2086
Epoch 6/50
17/17 [==============================] - 7s 381ms/step - loss: 2208.6309 - categorical_accuracy: 0.1705
Epoch 7/50
17/17 [==============================] - 7s 408ms/step - loss: 2437.0120 - categorical_accuracy: 0.1952
Epoch 8/50
17/17 [==============================] - 6s 373ms/step - loss: 2980.9355 - categorical_accuracy: 0.1943
Epoch 9/50
17/17 [==============================] - 6s 375ms/step - loss: 3627.92

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.argmax(y_pred, axis=1).tolist()

In [ ]:
y_true = np.argmax(y_test, axis=1).tolist()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,ConfusionMatrixDisplay, confusion_matrix

In [ ]:
confusion_matrices = confusion_matrix(y_true, y_pred,labels=np.unique(y_pred), normalize='true')

In [ ]:
confusion_matrices

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrices, display_labels=np.unique(y_pred))
disp.plot()